In [1]:
from surprise import Dataset
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
data = Dataset.load_builtin('ml-1m')
df = pd.DataFrame(data.raw_ratings)

In [3]:
users = df[0].unique()
items = df[1].unique()

In [4]:
class UserMovieEmbedding(nn.Module):
    def __init__(self, users_count, items_count, embedding_dim):
        super().__init__()
        self.user_embeddings = nn.Embedding(users_count, embedding_dim)
        self.item_embeddings = nn.Embedding(items_count, embedding_dim)
        
    def forward(self, user, items):
        user_embedding = self.user_embeddings(user)
        item_embeddings = self.item_embeddings(items)
        return [user_embedding, item_embeddings]

In [5]:
class DrrAveState(nn.Module):
    def __init__(self, embedding_dim):
        super().__init__()
        self.conv = torch.nn.Conv1d(in_channels=embedding_dim, out_channels=1, kernel_size=1)
        self.avg_pool = torch.nn.AvgPool1d(kernel_size=1)
        
    def forward(self, item_embeddings):
        drr_ave = self.conv(item_embeddings)
        drr_ave = self.avg_pool(drr_ave).squeeze(1)
        return torch.cat((user_embedding, user_embedding * drr_ave, drr_ave), 1)

In [6]:
user = 1

itemsForFirstUser = torch.LongTensor(df[df[0]==str(user)][1].to_numpy(dtype = float))
embedding_dim = len(itemsForFirstUser)

user_movie_embedding = UserMovieEmbedding(len(users), len(items), embedding_dim)
user_embedding, item_embeddings = user_movie_embedding(torch.LongTensor([user]),  torch.LongTensor(np.expand_dims(itemsForFirstUser, axis=0)))

drrAveState = DrrAveState(embedding_dim)
state = drrAveState(item_embeddings)

In [7]:
class Actor(nn.Module):
    
    def __init__(self, state_dim, hidden_dim, output_dim):
        super().__init__()        
        self.lin1 = nn.Linear(in_features=state_dim, out_features=hidden_dim)
        self.lin2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.lin3 = nn.Linear(in_features=hidden_dim, out_features=output_dim)

    def forward(self, state):
        new_state = torch.relu(self.lin1(state))
        new_state = torch.relu(self.lin2(new_state))
        action = torch.tanh(self.lin3(new_state))
        return action
    
def calc_ranking(item_embedding, actor_output):
    return torch.matmul(item_embedding, actor_output[0])


In [8]:
actor = Actor(len(state[0]), embedding_dim, embedding_dim)

In [9]:
action = actor(state)

In [10]:
item_embed = user_movie_embedding.item_embeddings(torch.LongTensor(np.array(df[df[0]==str(user)][1][0], dtype=float)))
calc_ranking(item_embed, action)

tensor(-1.5406, grad_fn=<DotBackward>)